<a href="https://colab.research.google.com/github/inggilove/demo-ai/blob/main/2020_12_MF_SVD_%EA%B8%B0%EC%B4%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 추천시스템 테스트

 - MovieLens 1M Dataset
    + MovieLens 1M movie ratings. Stable benchmark dataset. 
    + 1 million ratings from 6000 users on 4000 movies. 
    + Released 2/2003.
 - SVD 참조 : https://lsjsj92.tistory.com/570

 - 처음 만들어본 MF - SVD ( MovieLens 데이터 )
   > k 값을 조절하여 확률개선 ( 크다고 좋은 건 아니고, 75% 로 할 때 적절 )

In [17]:
import os
print(os.getcwd())
!ls
!cd /content
!ls

/content
MyDrive  sample_data
MyDrive  sample_data


In [19]:
from google.colab import drive
drive.mount('/MyDrive')

Mounted at /MyDrive


In [45]:
!cd "/MyDrive"
!ls -l "/MyDrive/My Drive/Colab Notebooks"

total 55
-rw------- 1 root root  1394 Dec 25 01:04  2020.12.kakao-buffalo.ipynb
-rw------- 1 root root 46521 Dec 25 22:40 '2020.12.kakao-buffalo.ipynb의 사본'
drwx------ 2 root root  4096 Dec 25 05:32  ml-1m
drwx------ 2 root root  4096 Dec 25 05:32  ml-25m


In [29]:
import pandas as pd
#df = pd.read_csv("./MyDrive/My Drive/Colab Notebooks/ml-1m/users.dat", sep='::',names=['UserID', 'MovieID', 'Rating', 'Timestamp'])
ratings = pd.read_csv("/MyDrive/My Drive/Colab Notebooks/ml-1m/ratings.dat", sep='::',names=['UserID', 'MovieID', 'Rating', 'Timestamp'])
movies = pd.read_csv("/MyDrive/My Drive/Colab Notebooks/ml-1m/movies.dat", sep='::',names=['MovieID', 'Title', 'Genres'])
users = pd.read_csv("/MyDrive/My Drive/Colab Notebooks/ml-1m/users.dat", sep='::',names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
#print(ratings)
#print(movies)
#print(users)
ratings.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning 

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [38]:
# Data 점검
ratings.groupby('UserID').count().describe()

,MovieID,Rating,Timestamp
count,6040.000000,6040.000000,6040.000000
mean,165.597517,165.597517,165.597517
std,192.747029,192.747029,192.747029
min,20.000000,20.000000,20.000000
25%,44.000000,44.000000,44.000000
50%,96.000000,96.000000,96.000000
75%,208.000000,208.000000,208.000000
max,2314.000000,2314.000000,2314.000000


In [30]:
# @2020.12.25, 
# User-Movie Rating Pivot & Normalization ( 평균값 제거 ) 
#
um_ratings = ratings.pivot( index = 'UserID', columns = 'MovieID', values = 'Rating').fillna(0)
#um_ratings.head()
matrix = um_ratings.values # as_matrix()

import numpy as np
ur_mean = np.mean(matrix, axis = 1)
matrix_u_mean = matrix - ur_mean.reshape(-1, 1)
pd.DataFrame( matrix_u_mean, columns = um_ratings.columns).head()

MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.940097,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,...,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903,-0.059903
1,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,0.870750,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,...,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250,-0.129250
2,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,...,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697,-0.053697
3,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,...,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745,-0.023745
4,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,1.831894,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,2.831894,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,0.831894,-0.168106,-0.168106,-0.168106,-0.168106,4.831894,-0.168106,-0.168106,3.831894,-0.168106,3.831894,-0.168106,2.831894,-0.168106,-0.168106,2.831894,-0.168106,...,-0.168106,-0.168106,-0.168106,-0.168106,-0.168106,-0.1681

In [32]:
print(matrix_u_mean.shape)
print( min(matrix_u_mean.shape))

(6040, 3706)
3706


In [49]:
#
# SVD
#

# (1) SVD
from scipy.sparse.linalg import svds, eigs

my_k = 200 # 1< k < min(A.shape), 75% 수준인 200개로 k 값 지정 ( Prediction = -1 수준 ), 1000 일 때 -0.65 )
U, sigma, Vt = svds( matrix_u_mean, k = my_k)
print(U.shape)
print(sigma.shape)
print(Vt.shape)
sigma = np.diag(sigma)
sigma[0]

# (2) SVD 특이값 분해를 사용한 MF 를 기반으로 데이터 변경
svd_user_pred_ratings = np.dot( np.dot(U, sigma), Vt ) + ur_mean.reshape(-1,1)
svd_preds = pd.DataFrame(svd_user_pred_ratings, columns = um_ratings.columns)
svd_preds.head()

(6040, 200)
(200,)
(200, 3706)


MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.438997,0.354180,0.270591,0.071100,-0.032643,-0.032715,-0.024866,0.007436,0.035225,-0.395815,0.190098,-0.041866,0.117933,-0.011857,-0.018418,-0.233187,0.596817,0.004957,0.261239,-0.023299,0.136459,0.185365,0.079925,0.208051,0.229325,0.076085,0.233101,0.054377,0.179898,-0.077356,0.122232,0.334900,-0.008540,0.600152,0.040925,0.263599,0.051203,0.014378,0.308638,0.025896,...,-0.004361,-0.100781,0.045494,0.207927,0.056449,0.089254,0.006234,-0.109771,0.036011,0.026574,0.008792,-0.000074,-0.076022,0.019805,-0.060083,-0.084821,0.024760,-0.038959,-0.021341,-0.094548,-0.004821,0.023081,0.016616,-0.120235,-0.010109,0.023930,0.019334,0.018179,0.036440,0.009752,0.053545,0.006166,0.026736,-0.103562,-0.147391,0.397532,0.033821,-0.051693,0.085548,-0.182831
1,0.033703,0.119277,0.472663,-0.000817,0.132402,0.622587,-0.299823,0.038745,0.133416,1.614002,-0.294640,0.126227,-0.029725,0.034284,0.046478,-0.022602,0.724885,-0.127494,-0.347483,0.080545,0.528766,0.196883,-0.117060,0.702170,-0.067417,0.147895,-0.150743,-0.280493,0.156511,0.185380,0.017497,-0.006440,-0.022520,0.412758,0.091744,-0.015695,-0.030498,-0.052068,0.437234,0.087879,...,0.011055,0.010025,-0.054227,0.051146,-0.005574,0.099848,-0.063339,-0.026628,-0.044515,-0.038287,0.054342,-0.044821,-0.173144,0.231932,0.304752,0.067977,0.059795,0.048673,-0.026705,0.140749,0.051386,0.058063,0.018972,0.103317,-0.105778,0.006693,0.017995,-0.022961,0.001044,-0.003033,-0.051370,0.010979,-0.043581,0.045930,0.071861,-0.775576,-0.051795,0.052210,0.078146,0.256389
2,0.814910,0.502804,0.033820,-0.078739,0.033998,0.162410,0.019663,0.041350,-0.052632,1.127378,-0.347980,0.067812,0.114939,-0.003569,0.183006,0.102722,0.047085,0.167194,-0.007680,0.106792,-0.507603,-0.125105,0.070462,0.029748,0.203586,0.081806,-0.034305,-0.187583,0.094616,-0.019460,0.004918,0.552458,-0.026311,-0.036000,0.059006,0.017924,-0.039156,-0.037934,0.087555,-0.034876,...,-0.023314,-0.051176,0.039305,-0.082416,-0.010470,-0.069817,-0.016369,-0.133019,0.039244,-0.001048,0.044485,0.044118,0.062186,-0.125941,0.001925,0.012515,-0.096184,0.093731,-0.008085,-0.000770,-0.005658,-0.034554,-0.032486,0.128240,0.013126,0.040218,-0.000635,0.000714,0.031652,0.016813,0.060226,-0.010422,0.023486,0.086383,0.019445,0.044415,0.343101,0.150808,0.005541,-0.051134
3,-0.415438,0.289797,-0.013074,0.091082,-0.061979,-0.194361,-0.091968,-0.005856,-0.026813,-0.250782,0.304277,0.037685,-0.010423,0.128374,-0.134672,-0.030617,0.084887,-0.004707,-0.070335,0.015047,0.218266,0.065582,-0.001540,0.144870,0.602724,0.040378,0.051818,-0.009523,0.184615,0.100921,0.017670,-0.226968,0.005911,0.094106,0.031277,0.160692,-0.024804,0.029563,-0.283699,0.023720,...,0.043723,-0.024401,0.043724,-0.064271,-0.114091,-0.023586,0.000957,-0.049837,-0.018569,-0.001993,-0.034343,-0.009890,-0.041346,-0.069240,-0.161412,-0.055978,-0.110248,-0.140191,-0.005871,-0.082806,0.016229,-0.004107,0.020703,-0.092832,-0.178418,0.004400,-0.011250,-0.006019,-0.002036,-0.009925,-0.005889,0.016387,0.015030,-0.034509,-0.023210,0.118576,-0.086609,-0.053922,0.009408,-0.090798
4,-0.446804,-0.483294,0.114056,0.120665,-0.079081,0.659606,-0.192993,0.068992,-0.155124,-0.100587,-0.402777,-0.102428,0.005268,0.294294,0.088981,1.690825,1.418324,0.162360,-0.237051,-0.044574,0.976371,-0.094873,-0.202098,-0.234483,1.257873,0.160443,-0.044555,0.352918,1.888351,0.288896,-0.077543,3.874218,0.034189,3.680510,0.309211,1.807893,0.043978,-0.018609,2.110536,0.093867,...,0.089829,0.754507,0.036689,-0.547216,0.413878,0.048334,0.013191,0.210778,-0.060182,-0.087019,0.000963,-0.075735,0.105206,-0.236627,0.107179,0.429790,-0.090764,0.233206,0.034693,0.030533,0.014593,0.038743,0.052605,-0.081690,-0.137718,0.057441,0.045193

In [50]:
#
# 추천
#
def recommend ( df_svd_preds, userid, ori_movies_df, ori_ratings_df, num_recommend=5):
  user_row_no = userid - 1
  sorted_user_pred = df_svd_preds.iloc[ user_row_no ].sort_values(ascending=False)

  user_data = ori_ratings_df[ ori_ratings_df.UserID == userid ]
  user_history = user_data.merge( ori_movies_df, on = 'MovieID' ).sort_values(['Rating'], ascending=False)

  # 기존 등급 데이터 제거
  recommendations = ori_movies_df[ ~ori_movies_df['MovieID'].isin( user_history['MovieID']) ]
  recommendations = recommendations.merge( pd.DataFrame( sorted_user_pred ).reset_index(), on = 'MovieID')
  recommendations = recommendations.rename( columns = {user_row_no: 'Predictions'}).sort_values('Predictions')

  return user_history, recommendations

In [51]:
#
# 추천 테스트
#
old, new = recommend( svd_preds, 330, movies, ratings, 10)
#old.head(10)
new.head(10)

,MovieID,Title,Genres,Predictions
1356,1500,Grosse Pointe Blank (1997),Comedy|Crime,-1.340943
2272,2502,Office Space (1999),Comedy|Romance,-1.090907
2023,2248,Say Anything... (1989),Comedy|Drama|Romance,-0.925768
1417,1573,Face/Off (1997),Action|Sci-Fi|Thriller,-0.866622
1517,1682,"Truman Show, The (1998)",Drama,-0.810506
1849,2064,Roger & Me (1989),Comedy|Documentary,-0.713425
2018,2243,Broadcast News (1987),Comedy|Drama|Romance,-0.653006
1706,1914,Smoke Signals (1998),Comedy|Drama,-0.639585
692,733,"Rock, The (1996)",Action|Adventure|Thriller,-0.626009
1743,1954,Rocky (1976),Action|Drama,-0.609442
